In [32]:
from pipeline import *

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config
from sklearn import metrics

np.random.seed(101)
pd.set_option('display.max_columns', None)
set_config(transform_output="pandas")

In [34]:
df = pd.read_parquet(r'D:\Project\MLO\data\phase-2\prob-1\raw_train.parquet')

In [35]:
df = df.sample(frac=1)

In [36]:
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label
34187,0.081963,tcp,-,FIN,22.0,14.0,1470.0,1728.0,31.0,29.0,1.370374e+05,156656.046900,5.0,4.0,3.903000,6.248692,399.950226,14.982982,255.0,1.213830e+09,3.363719e+09,255.0,0.000707,0.000582,0.000125,67.0,123.0,0.0,0.0,9.0,0.0,4.0,1.0,1.0,3.0,0.0,0.0,0.0,3.0,16.0,0.0,0
28653,0.000010,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,4.560000e+07,0.000000,0.0,0.0,0.010000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,4.0,2.0,5.0,4.0,4.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,1
7570,0.796955,tcp,-,FIN,10.0,10.0,608.0,646.0,254.0,252.0,5.500938e+03,5842.237305,2.0,3.0,88.550556,80.579555,4858.346581,127.869945,255.0,2.375786e+09,2.121203e+09,255.0,0.159596,0.070978,0.088618,61.0,65.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,1
50165,0.000009,udp,dns,INT,2.0,0.0,114.0,0.0,254.0,0.0,5.066666e+07,0.000000,0.0,0.0,0.009000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000,0.000000,57.0,0.0,0.0,0.0,33.0,2.0,26.0,26.0,13.0,33.0,0.0,0.0,0.0,26.0,33.0,0.0,1
55555,0.376387,tcp,-,FIN,10.0,6.0,1612.0,268.0,254.0,252.0,3.084060e+04,4761.057129,2.0,1.0,38.067556,59.815801,2007.245167,88.242922,255.0,1.663055e+09,3.271902e+08,255.0,0.136073,0.077299,0.058774,161.0,45.0,0.0,0.0,7.0,1.0,2.0,2.0,1.0,7.0,0.0,0.0,0.0,2.0,7.0,0.0,1


In [37]:
fins = ['feature2', 'feature3', 'feature16', 'feature27', 'feature7',
       'feature26', 'feature35', 'feature6', 'feature40', 'feature32',
       'feature1', 'feature11', 'feature33', 'feature17', 'feature39',
       'feature18', 'feature30', 'feature15', 'feature21']

nums = [
    "feature1",    "feature5",    "feature6",    "feature7",    "feature8",    "feature9",    "feature10",    "feature11",
    "feature12",    "feature13",    "feature14",    "feature15",    "feature16",    "feature17",    "feature18",    "feature19",
    "feature20",    "feature21",    "feature22",    "feature23",    "feature24",    "feature25",    "feature26",    "feature27",
    "feature28",    "feature29",    "feature30",    "feature31",    "feature32",    "feature33",    "feature34",    "feature35",
    "feature36",    "feature37",    "feature38",    "feature39",    "feature40",    "feature41"
]

cats = [
    "feature2", "feature3", "feature4"
]

nums = [i for i in nums if i in fins]
cats = [i for i in cats if i in fins]

In [38]:
import re

nums = [int(re.findall(r'\d+', num)[0]) - 1 for num in nums]
cats = [int(re.findall(r'\d+', cat)[0]) - 1 for cat in cats]
fins = nums + cats

In [39]:
X_train, X_val, y_train, y_val = train_test_split(df.drop(columns=['label']).values, df['label'].values, test_size=0.2, random_state=101)

In [40]:
from sklearn.pipeline import FeatureUnion, Pipeline

In [41]:
s = Pipeline(
    steps=(
        ('woe', WOE(cats=cats, nums=[])),
        ('feature_select', FeatureSelector(fins))
    )
)

In [42]:
s.fit(X_train, y_train)

Pipeline(steps=[('woe', WOE(cats=[1, 2], nums=[])),
                ('feature_select',
                 FeatureSelector(_feature_names=[0, 5, 6, 10, 14, 15, 16, 17,
                                                 20, 25, 26, 29, 31, 32, 34, 38,
                                                 39, 1, 2]))])

In [43]:
import pickle

with open(r'D:\Project\MLO\src\phase2\models\pipeline_1.pkl', 'wb') as f:
    pickle.dump(s, f)

In [44]:
%time
X_train = s.transform(X_train)
X_val = s.transform(X_val)

CPU times: total: 0 ns
Wall time: 0 ns


In [45]:
X_train

array([[4e-06, 0.0, 114.0, ..., 33.0, 0.0, 0.0],
       [6e-06, 0.0, 114.0, ..., 19.0, 0.0, 0.0],
       [1.1e-05, 0.0, 114.0, ..., 14.0, 0.0, 0.0],
       ...,
       [49.62455, 0.0, 2138.0, ..., 6.0, 0.0, 0.0],
       [1.500286, 18.0, 117987.0, ..., 3.0, 0.0, 0.0],
       [0.005072, 20.0, 2158.0, ..., 7.0, 0.0, 0.0]], dtype=object)

In [46]:
model = CatBoostClassifier(
    iterations=2000,
    random_seed=101,
    learning_rate=0.02,
    eval_metric='AUC',
    use_best_model=True
)

In [47]:
model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	test: 0.9305686	best: 0.9305686 (0)	total: 16.5ms	remaining: 32.9s
1:	test: 0.9469710	best: 0.9469710 (1)	total: 28.5ms	remaining: 28.5s
2:	test: 0.9514502	best: 0.9514502 (2)	total: 40.9ms	remaining: 27.2s
3:	test: 0.9522777	best: 0.9522777 (3)	total: 51.1ms	remaining: 25.5s
4:	test: 0.9539476	best: 0.9539476 (4)	total: 62.1ms	remaining: 24.8s
5:	test: 0.9578775	best: 0.9578775 (5)	total: 74.3ms	remaining: 24.7s
6:	test: 0.9580629	best: 0.9580629 (6)	total: 85.4ms	remaining: 24.3s
7:	test: 0.9585340	best: 0.9585340 (7)	total: 96.3ms	remaining: 24s
8:	test: 0.9609563	best: 0.9609563 (8)	total: 106ms	remaining: 23.5s
9:	test: 0.9636403	best: 0.9636403 (9)	total: 117ms	remaining: 23.2s
10:	test: 0.9632486	best: 0.9636403 (9)	total: 127ms	remaining: 23s
11:	test: 0.9647922	best: 0.9647922 (11)	total: 138ms	remaining: 22.8s
12:	test: 0.9645400	best: 0.9647922 (11)	total: 148ms	remaining: 22.6s
13:	test: 0.9648502	best: 0.9648502 (13)	total: 158ms	remaining: 22.4s
14:	test: 0.9657518	bes

128:	test: 0.9808281	best: 0.9808281 (128)	total: 1.29s	remaining: 18.8s
129:	test: 0.9808363	best: 0.9808363 (129)	total: 1.3s	remaining: 18.8s
130:	test: 0.9808684	best: 0.9808684 (130)	total: 1.31s	remaining: 18.8s
131:	test: 0.9809201	best: 0.9809201 (131)	total: 1.32s	remaining: 18.7s
132:	test: 0.9809512	best: 0.9809512 (132)	total: 1.33s	remaining: 18.7s
133:	test: 0.9809807	best: 0.9809807 (133)	total: 1.34s	remaining: 18.7s
134:	test: 0.9809897	best: 0.9809897 (134)	total: 1.35s	remaining: 18.6s
135:	test: 0.9809897	best: 0.9809897 (134)	total: 1.37s	remaining: 18.8s
136:	test: 0.9810258	best: 0.9810258 (136)	total: 1.38s	remaining: 18.8s
137:	test: 0.9810806	best: 0.9810806 (137)	total: 1.39s	remaining: 18.7s
138:	test: 0.9811122	best: 0.9811122 (138)	total: 1.4s	remaining: 18.7s
139:	test: 0.9811339	best: 0.9811339 (139)	total: 1.4s	remaining: 18.7s
140:	test: 0.9811499	best: 0.9811499 (140)	total: 1.42s	remaining: 18.7s
141:	test: 0.9812054	best: 0.9812054 (141)	total: 1.42

248:	test: 0.9839095	best: 0.9839095 (248)	total: 2.38s	remaining: 16.7s
249:	test: 0.9839295	best: 0.9839295 (249)	total: 2.39s	remaining: 16.7s
250:	test: 0.9839517	best: 0.9839517 (250)	total: 2.4s	remaining: 16.7s
251:	test: 0.9839781	best: 0.9839781 (251)	total: 2.4s	remaining: 16.7s
252:	test: 0.9839912	best: 0.9839912 (252)	total: 2.41s	remaining: 16.7s
253:	test: 0.9839975	best: 0.9839975 (253)	total: 2.42s	remaining: 16.6s
254:	test: 0.9840057	best: 0.9840057 (254)	total: 2.43s	remaining: 16.6s
255:	test: 0.9840202	best: 0.9840202 (255)	total: 2.44s	remaining: 16.6s
256:	test: 0.9840361	best: 0.9840361 (256)	total: 2.45s	remaining: 16.6s
257:	test: 0.9840415	best: 0.9840415 (257)	total: 2.46s	remaining: 16.6s
258:	test: 0.9840584	best: 0.9840584 (258)	total: 2.46s	remaining: 16.6s
259:	test: 0.9840676	best: 0.9840676 (259)	total: 2.47s	remaining: 16.5s
260:	test: 0.9840939	best: 0.9840939 (260)	total: 2.48s	remaining: 16.5s
261:	test: 0.9841080	best: 0.9841080 (261)	total: 2.4

365:	test: 0.9853990	best: 0.9853990 (365)	total: 3.4s	remaining: 15.2s
366:	test: 0.9854221	best: 0.9854221 (366)	total: 3.41s	remaining: 15.2s
367:	test: 0.9854306	best: 0.9854306 (367)	total: 3.42s	remaining: 15.2s
368:	test: 0.9854391	best: 0.9854391 (368)	total: 3.43s	remaining: 15.2s
369:	test: 0.9854508	best: 0.9854508 (369)	total: 3.44s	remaining: 15.1s
370:	test: 0.9854662	best: 0.9854662 (370)	total: 3.44s	remaining: 15.1s
371:	test: 0.9854672	best: 0.9854672 (371)	total: 3.45s	remaining: 15.1s
372:	test: 0.9854702	best: 0.9854702 (372)	total: 3.46s	remaining: 15.1s
373:	test: 0.9854873	best: 0.9854873 (373)	total: 3.47s	remaining: 15.1s
374:	test: 0.9854883	best: 0.9854883 (374)	total: 3.48s	remaining: 15.1s
375:	test: 0.9855088	best: 0.9855088 (375)	total: 3.49s	remaining: 15.1s
376:	test: 0.9855160	best: 0.9855160 (376)	total: 3.5s	remaining: 15.1s
377:	test: 0.9855214	best: 0.9855214 (377)	total: 3.5s	remaining: 15s
378:	test: 0.9855332	best: 0.9855332 (378)	total: 3.51s	

483:	test: 0.9863147	best: 0.9863147 (483)	total: 4.68s	remaining: 14.7s
484:	test: 0.9863257	best: 0.9863257 (484)	total: 4.7s	remaining: 14.7s
485:	test: 0.9863311	best: 0.9863311 (485)	total: 4.71s	remaining: 14.7s
486:	test: 0.9863328	best: 0.9863328 (486)	total: 4.72s	remaining: 14.7s
487:	test: 0.9863433	best: 0.9863433 (487)	total: 4.73s	remaining: 14.7s
488:	test: 0.9863534	best: 0.9863534 (488)	total: 4.74s	remaining: 14.7s
489:	test: 0.9863616	best: 0.9863616 (489)	total: 4.75s	remaining: 14.7s
490:	test: 0.9863675	best: 0.9863675 (490)	total: 4.77s	remaining: 14.7s
491:	test: 0.9863698	best: 0.9863698 (491)	total: 4.78s	remaining: 14.6s
492:	test: 0.9863750	best: 0.9863750 (492)	total: 4.79s	remaining: 14.6s
493:	test: 0.9863949	best: 0.9863949 (493)	total: 4.8s	remaining: 14.6s
494:	test: 0.9864020	best: 0.9864020 (494)	total: 4.81s	remaining: 14.6s
495:	test: 0.9864044	best: 0.9864044 (495)	total: 4.83s	remaining: 14.6s
496:	test: 0.9864102	best: 0.9864102 (496)	total: 4.8

...

1544:	test: 0.9890640	best: 0.9890640 (1544)	total: 26.2s	remaining: 7.71s
1545:	test: 0.9890626	best: 0.9890640 (1544)	total: 26.2s	remaining: 7.69s
1546:	test: 0.9890637	best: 0.9890640 (1544)	total: 26.2s	remaining: 7.68s
1547:	test: 0.9890627	best: 0.9890640 (1544)	total: 26.2s	remaining: 7.66s
1548:	test: 0.9890630	best: 0.9890640 (1544)	total: 26.3s	remaining: 7.65s
1549:	test: 0.9890651	best: 0.9890651 (1549)	total: 26.3s	remaining: 7.63s
1550:	test: 0.9890657	best: 0.9890657 (1550)	total: 26.3s	remaining: 7.61s
1551:	test: 0.9890659	best: 0.9890659 (1551)	total: 26.3s	remaining: 7.59s
1552:	test: 0.9890669	best: 0.9890669 (1552)	total: 26.3s	remaining: 7.58s
1553:	test: 0.9890674	best: 0.9890674 (1553)	total: 26.3s	remaining: 7.56s
1554:	test: 0.9890692	best: 0.9890692 (1554)	total: 26.4s	remaining: 7.54s
1555:	test: 0.9890685	best: 0.9890692 (1554)	total: 26.4s	remaining: 7.53s
1556:	test: 0.9890783	best: 0.9890783 (1556)	total: 26.4s	remaining: 7.52s
1557:	test: 0.9890765	bes

1656:	test: 0.9891198	best: 0.9891204 (1655)	total: 28.4s	remaining: 5.87s
1657:	test: 0.9891223	best: 0.9891223 (1657)	total: 28.4s	remaining: 5.86s
1658:	test: 0.9891243	best: 0.9891243 (1658)	total: 28.4s	remaining: 5.84s
1659:	test: 0.9891230	best: 0.9891243 (1658)	total: 28.4s	remaining: 5.83s
1660:	test: 0.9891214	best: 0.9891243 (1658)	total: 28.5s	remaining: 5.81s
1661:	test: 0.9891225	best: 0.9891243 (1658)	total: 28.5s	remaining: 5.79s
1662:	test: 0.9891230	best: 0.9891243 (1658)	total: 28.5s	remaining: 5.78s
1663:	test: 0.9891211	best: 0.9891243 (1658)	total: 28.5s	remaining: 5.76s
1664:	test: 0.9891212	best: 0.9891243 (1658)	total: 28.6s	remaining: 5.75s
1665:	test: 0.9891229	best: 0.9891243 (1658)	total: 28.6s	remaining: 5.73s
1666:	test: 0.9891221	best: 0.9891243 (1658)	total: 28.6s	remaining: 5.71s
1667:	test: 0.9891225	best: 0.9891243 (1658)	total: 28.6s	remaining: 5.7s
1668:	test: 0.9891221	best: 0.9891243 (1658)	total: 28.6s	remaining: 5.68s
1669:	test: 0.9891210	best

1768:	test: 0.9891742	best: 0.9891755 (1767)	total: 30.8s	remaining: 4.02s
1769:	test: 0.9891756	best: 0.9891756 (1769)	total: 30.8s	remaining: 4s
1770:	test: 0.9891764	best: 0.9891764 (1770)	total: 30.8s	remaining: 3.98s
1771:	test: 0.9891776	best: 0.9891776 (1771)	total: 30.8s	remaining: 3.97s
1772:	test: 0.9891803	best: 0.9891803 (1772)	total: 30.8s	remaining: 3.95s
1773:	test: 0.9891810	best: 0.9891810 (1773)	total: 30.9s	remaining: 3.93s
1774:	test: 0.9891798	best: 0.9891810 (1773)	total: 30.9s	remaining: 3.92s
1775:	test: 0.9891812	best: 0.9891812 (1775)	total: 30.9s	remaining: 3.9s
1776:	test: 0.9891812	best: 0.9891812 (1775)	total: 30.9s	remaining: 3.88s
1777:	test: 0.9891837	best: 0.9891837 (1777)	total: 30.9s	remaining: 3.86s
1778:	test: 0.9891857	best: 0.9891857 (1778)	total: 31s	remaining: 3.85s
1779:	test: 0.9891858	best: 0.9891858 (1779)	total: 31s	remaining: 3.83s
1780:	test: 0.9891836	best: 0.9891858 (1779)	total: 31s	remaining: 3.81s
1781:	test: 0.9891862	best: 0.98918

1885:	test: 0.9892227	best: 0.9892293 (1870)	total: 33.1s	remaining: 2s
1886:	test: 0.9892216	best: 0.9892293 (1870)	total: 33.2s	remaining: 1.99s
1887:	test: 0.9892224	best: 0.9892293 (1870)	total: 33.2s	remaining: 1.97s
1888:	test: 0.9892222	best: 0.9892293 (1870)	total: 33.2s	remaining: 1.95s
1889:	test: 0.9892200	best: 0.9892293 (1870)	total: 33.2s	remaining: 1.93s
1890:	test: 0.9892221	best: 0.9892293 (1870)	total: 33.3s	remaining: 1.92s
1891:	test: 0.9892228	best: 0.9892293 (1870)	total: 33.3s	remaining: 1.9s
1892:	test: 0.9892237	best: 0.9892293 (1870)	total: 33.3s	remaining: 1.88s
1893:	test: 0.9892231	best: 0.9892293 (1870)	total: 33.3s	remaining: 1.86s
1894:	test: 0.9892223	best: 0.9892293 (1870)	total: 33.3s	remaining: 1.85s
1895:	test: 0.9892251	best: 0.9892293 (1870)	total: 33.4s	remaining: 1.83s
1896:	test: 0.9892263	best: 0.9892293 (1870)	total: 33.4s	remaining: 1.81s
1897:	test: 0.9892277	best: 0.9892293 (1870)	total: 33.4s	remaining: 1.8s
1898:	test: 0.9892291	best: 0.

1998:	test: 0.9892687	best: 0.9892687 (1998)	total: 35.5s	remaining: 17.8ms
1999:	test: 0.9892700	best: 0.9892700 (1999)	total: 35.6s	remaining: 0us

bestTest = 0.9892700477
bestIteration = 1999



In [48]:
metrics.roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

0.9892700476710478

In [49]:
# vl = pd.DataFrame([X_train.columns, model.get_feature_importance()]).T
# vl.columns = ['Feature', 'Importance']
# vl = vl.sort_values('Importance', ascending=False)
# vl

In [50]:
import pickle

with open(r'D:\Project\MLO\src\phase2\models\problem1.pkl', 'wb') as f:
    pickle.dump(model, f)

In [11]:
s = [
        [
            0.081963,
            "tcp",
            "-",
            "FIN",
            22.0,
            14.0,
            1470.0,
            1728.0,
            31.0,
            29.0,
            137037.4375,
            156656.0469,
            5.0,
            4.0,
            3.903,
            6.248692,
            399.950226,
            14.982982,
            255.0,
            1213829701.0,
            3363718728.0,
            255.0,
            0.000707,
            0.000582,
            0.000125,
            67.0,
            123.0,
            0.0,
            0.0,
            9.0,
            0.0,
            4.0,
            1.0,
            1.0,
            3.0,
            0.0,
            0.0,
            0.0,
            3.0,
            16.0,
            0.0
        ],
        [
            1e-05,
            "udp",
            "dns",
            "INT",
            2.0,
            0.0,
            114.0,
            0.0,
            254.0,
            0.0,
            45600000.0,
            0.0,
            0.0,
            0.0,
            0.01,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            57.0,
            0.0,
            0.0,
            0.0,
            4.0,
            2.0,
            5.0,
            4.0,
            4.0,
            4.0,
            0.0,
            0.0,
            0.0,
            4.0,
            4.0,
            0.0
        ],
        [
            0.796955,
            "tcp",
            "-",
            "FIN",
            10.0,
            10.0,
            608.0,
            646.0,
            254.0,
            252.0,
            5500.937988,
            5842.237305,
            2.0,
            3.0,
            88.550556,
            80.579555,
            4858.346581,
            127.869945,
            255.0,
            2375785929.0,
            2121203199.0,
            255.0,
            0.159596,
            0.070978,
            0.088618,
            61.0,
            65.0,
            0.0,
            0.0,
            1.0,
            1.0,
            2.0,
            1.0,
            1.0,
            1.0,
            0.0,
            0.0,
            0.0,
            1.0,
            3.0,
            0.0
        ],
        [
            9e-06,
            "udp",
            "dns",
            "INT",
            2.0,
            0.0,
            114.0,
            0.0,
            254.0,
            0.0,
            50666664.0,
            0.0,
            0.0,
            0.0,
            0.009,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            57.0,
            0.0,
            0.0,
            0.0,
            33.0,
            2.0,
            26.0,
            26.0,
            13.0,
            33.0,
            0.0,
            0.0,
            0.0,
            26.0,
            33.0,
            0.0
        ],
        [
            0.376387,
            "tcp",
            "-",
            "FIN",
            10.0,
            6.0,
            1612.0,
            268.0,
            254.0,
            252.0,
            30840.59766,
            4761.057129,
            2.0,
            1.0,
            38.067556,
            59.815801,
            2007.245167,
            88.242922,
            255.0,
            1663054526.0,
            327190215.0,
            255.0,
            0.136073,
            0.077299,
            0.058774,
            161.0,
            45.0,
            0.0,
            0.0,
            7.0,
            1.0,
            2.0,
            2.0,
            1.0,
            7.0,
            0.0,
            0.0,
            0.0,
            2.0,
            7.0,
            0.0
        ],
        [
            0.539447,
            "tcp",
            "-",
            "FIN",
            12.0,
            26.0,
            790.0,
            29462.0,
            62.0,
            252.0,
            10751.75098,
            420119.125,
            2.0,
            11.0,
            49.040636,
            20.82488,
            3857.955887,
            2883.649032,
            255.0,
            2085586880.0,
            2746068499.0,
            255.0,
            0.033278,
            0.018104,
            0.015174,
            66.0,
            1133.0,
            0.0,
            0.0,
            2.0,
            1.0,
            1.0,
            1.0,
            1.0,
            1.0,
            0.0,
            0.0,
            0.0,
            1.0,
            1.0,
            0.0
        ],
        [
            0.948628,
            "tcp",
            "http",
            "FIN",
            12.0,
            10.0,
            1180.0,
            2838.0,
            254.0,
            252.0,
            9124.756836,
            21546.9082,
            3.0,
            3.0,
            86.238906,
            92.182445,
            4584.031229,
            147.667359,
            255.0,
            3307308849.0,
            2594870149.0,
            255.0,
            0.134007,
            0.048976,
            0.085031,
            98.0,
            284.0,
            1.0,
            457.0,
            5.0,
            1.0,
            1.0,
            3.0,
            1.0,
            1.0,
            0.0,
            0.0,
            1.0,
            3.0,
            1.0,
            0.0
        ],
        [
            58.223106,
            "udp",
            "-",
            "CON",
            300.0,
            300.0,
            187688.0,
            236626.0,
            254.0,
            60.0,
            25702.92188,
            32404.72852,
            0.0,
            0.0,
            194.399859,
            194.367094,
            239.880531,
            242.059672,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            626.0,
            789.0,
            0.0,
            0.0,
            2.0,
            0.0,
            1.0,
            1.0,
            1.0,
            1.0,
            0.0,
            0.0,
            0.0,
            1.0,
            1.0,
            0.0
        ],
        [
            0.811656,
            "tcp",
            "-",
            "FIN",
            10.0,
            6.0,
            588.0,
            268.0,
            254.0,
            252.0,
            5223.888184,
            2207.831787,
            2.0,
            1.0,
            77.140222,
            144.962203,
            3897.384006,
            198.674094,
            255.0,
            4098859561.0,
            728086780.0,
            255.0,
            0.215227,
            0.086842,
            0.128385,
            59.0,
            45.0,
            0.0,
            0.0,
            3.0,
            1.0,
            1.0,
            1.0,
            1.0,
            2.0,
            0.0,
            0.0,
            0.0,
            2.0,
            2.0,
            0.0
        ],
        [
            0.001019,
            "udp",
            "dns",
            "CON",
            2.0,
            2.0,
            146.0,
            178.0,
            31.0,
            29.0,
            573110.9375,
            698724.25,
            0.0,
            0.0,
            0.009,
            0.004,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            0.0,
            73.0,
            89.0,
            0.0,
            0.0,
            3.0,
            0.0,
            8.0,
            1.0,
            1.0,
            4.0,
            0.0,
            0.0,
            0.0,
            5.0,
            5.0,
            0.0
        ]
    ]

In [17]:
s * 100

[[0.081963,
  'tcp',
  '-',
  'FIN',
  22.0,
  14.0,
  1470.0,
  1728.0,
  31.0,
  29.0,
  137037.4375,
  156656.0469,
  5.0,
  4.0,
  3.903,
  6.248692,
  399.950226,
  14.982982,
  255.0,
  1213829701.0,
  3363718728.0,
  255.0,
  0.000707,
  0.000582,
  0.000125,
  67.0,
  123.0,
  0.0,
  0.0,
  9.0,
  0.0,
  4.0,
  1.0,
  1.0,
  3.0,
  0.0,
  0.0,
  0.0,
  3.0,
  16.0,
  0.0],
 [1e-05,
  'udp',
  'dns',
  'INT',
  2.0,
  0.0,
  114.0,
  0.0,
  254.0,
  0.0,
  45600000.0,
  0.0,
  0.0,
  0.0,
  0.01,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  57.0,
  0.0,
  0.0,
  0.0,
  4.0,
  2.0,
  5.0,
  4.0,
  4.0,
  4.0,
  0.0,
  0.0,
  0.0,
  4.0,
  4.0,
  0.0],
 [0.796955,
  'tcp',
  '-',
  'FIN',
  10.0,
  10.0,
  608.0,
  646.0,
  254.0,
  252.0,
  5500.937988,
  5842.237305,
  2.0,
  3.0,
  88.550556,
  80.579555,
  4858.346581,
  127.869945,
  255.0,
  2375785929.0,
  2121203199.0,
  255.0,
  0.159596,
  0.070978,
  0.088618,
  61.0,
  65.0,
  0.0,
  0.0,
  1